In [ ]:
code='''
# ✅ Streamlit Smart Trading Dashboard (Interactive & Enhanced)
import streamlit as st
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from SmartApi import SmartConnect
import pyotp
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import requests
import time
import plotly.graph_objects as go
from deap import base, creator, tools, algorithms
import random
from textblob import TextBlob
import yfinance as yf  # For paper trading simulation

# Helper functions moved to the top
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def compute_macd(series, fast=12, slow=26, signal=9):
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

st.set_page_config(layout="wide")
st.title("📈 Smart Trading Dashboard with ML, Deep Learning, Backtest, Alerts & Auto-refresh")

# === Initialize Session State ===
if 'portfolio' not in st.session_state:
    st.session_state.portfolio = {}
if 'watchlist' not in st.session_state:
    st.session_state.watchlist = []
if 'community_trades' not in st.session_state:
    st.session_state.community_trades = []

# === Smart API Login Section ===
with st.expander("🔐 Smart API Login"):
    API_KEY = st.text_input("API Key", type="password")
    CLIENT_ID = st.text_input("Client ID")
    PASSWORD = st.text_input("Password", type="password")
    TOTP_SECRET = st.text_input("TOTP Secret (Base32)", type="password")

    if st.button("Login & Fetch Token"):
        try:
            totp = pyotp.TOTP(TOTP_SECRET)
            otp_code = totp.now()
            obj = SmartConnect(api_key=API_KEY)
            session_data = obj.generateSession(CLIENT_ID, PASSWORD, otp_code)
            access_token = session_data['data']['jwtToken']
            st.success("🔓 Logged in successfully!")
            st.session_state.obj = obj
        except Exception as e:
            st.error(f"Login failed: {e}")

# === Symbol Search & Selection ===
st.sidebar.header("🔍 Search & Select Symbol")
selected_symbol = None

try:
    scrip_url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
    scrip_data = pd.read_json(scrip_url)
    scrip_data['label'] = scrip_data['name'] + " (" + scrip_data['symbol'] + ")"
    query = st.sidebar.text_input("Type to search symbol e.g., RELI, BANK, TCS")
    if query:
        filtered = scrip_data[scrip_data['symbol'].str.contains(query.upper()) | scrip_data['name'].str.contains(query, case=False)]
        selected_label = st.sidebar.selectbox("Select from matches", filtered['label'].tolist())
        selected_symbol = filtered[filtered['label'] == selected_label].iloc[0]

        # Add to watchlist
        if st.sidebar.button("➕ Add to Watchlist"):
            if selected_symbol['symbol'] not in [s['symbol'] for s in st.session_state.watchlist]:
                st.session_state.watchlist.append(selected_symbol.to_dict())
                st.sidebar.success(f"Added {selected_symbol['symbol']} to watchlist")
except:
    st.sidebar.warning("Could not load symbol list. Check internet.")

# === Watchlist Management ===
if st.session_state.watchlist:
    st.sidebar.header("👀 Watchlist")
    for idx, symbol in enumerate(st.session_state.watchlist):
        cols = st.sidebar.columns([3,1])
        cols[0].write(f"{symbol['name']} ({symbol['symbol']})")
        if cols[1].button("❌", key=f"del_{idx}"):
            st.session_state.watchlist.pop(idx)
            st.experimental_rerun()

# === Historical Data Fetch Section ===
with st.expander("📥 Fetch Historical Data"):
    if 'obj' in st.session_state and selected_symbol is not None:
        timeframe_options = ['ONE_MINUTE', 'FIVE_MINUTE', 'FIFTEEN_MINUTE', 'THIRTY_MINUTE',
                            'ONE_HOUR', 'ONE_DAY', 'ONE_WEEK']
        selected_timeframe = st.selectbox("Select Timeframe", timeframe_options, index=4)

        col1, col2 = st.columns(2)
        from_date = col1.date_input("From Date", datetime.now() - timedelta(days=30))
        to_date = col2.date_input("To Date", datetime.now())

        if st.button("Fetch Data"):
            try:
                historic_data_params = {
                    'exchange': selected_symbol['exch_seg'],
                    'symboltoken': selected_symbol['token'],
                    'interval': selected_timeframe,
                    'fromdate': from_date.strftime("%Y-%m-%d 09:15"),
                    'todate': to_date.strftime("%Y-%m-%d 15:30")
                }
                historical_data = st.session_state.obj.getCandleData(historicDataParams=historic_data_params)
                df = pd.DataFrame(historical_data['data'], columns=["Time Interval", "Open Price", "High Price", "Low Price", "Close Price", "Volume"])
                df["Time Interval"] = pd.to_datetime(df["Time Interval"])
                df.set_index("Time Interval", inplace=True)
                st.session_state.df = df
                st.session_state.current_symbol = selected_symbol['symbol']
                st.success("✅ Data Fetched Successfully!")
                st.dataframe(df.tail(10))
            except Exception as e:
                st.error(f"Error fetching data: {e}")
    else:
        st.info("Login and select a symbol to fetch data.")

# === Upload CSV as alternate ===
st.sidebar.header("📤 Upload historical_data.csv")
uploaded_file = st.sidebar.file_uploader("Upload historical_data.csv", type="csv")
if uploaded_file:
    st.session_state.df = pd.read_csv(uploaded_file)
    st.session_state.df["Time Interval"] = pd.to_datetime(st.session_state.df["Time Interval"])
    st.session_state.df.set_index("Time Interval", inplace=True)
    st.success("✅ Data uploaded from CSV!")

# === Sentiment Analysis ===
with st.expander("📰 News Sentiment Analysis"):
    if 'current_symbol' in st.session_state:
        symbol = st.session_state.current_symbol
        st.write(f"Fetching news sentiment for {symbol}")

        # Simulated news sentiment (in a real app, you'd use a news API)
        sample_news = [
            f"{symbol} reports strong quarterly earnings",
            f"Market uncertainty affects {symbol} shares",
            f"Analysts upgrade {symbol} to buy rating",
            f"{symbol} faces regulatory challenges",
            f"New product launch boosts {symbol} outlook"
        ]

        sentiments = []
        for news in sample_news:
            analysis = TextBlob(news)
            sentiment = analysis.sentiment.polarity
            sentiments.append((news, sentiment))

            # Display sentiment with color coding
            if sentiment > 0.1:
                st.success(f"👍 {news} (Score: {sentiment:.2f})")
            elif sentiment < -0.1:
                st.error(f"👎 {news} (Score: {sentiment:.2f})")
            else:
                st.info(f"😐 {news} (Score: {sentiment:.2f})")

        avg_sentiment = np.mean([s[1] for s in sentiments])
        st.metric("Average News Sentiment Score", f"{avg_sentiment:.2f}",
                 delta="Positive" if avg_sentiment > 0 else "Negative")

# === Portfolio Simulation ===
with st.expander("📊 Portfolio Simulation"):
    st.write("Track multiple symbols in your portfolio")

    if 'df' in st.session_state and 'current_symbol' in st.session_state:
        symbol = st.session_state.current_symbol
        current_price = st.session_state.df['Close Price'].iloc[-1]

        col1, col2 = st.columns(2)
        with col1:
            buy_amount = st.number_input(f"Amount to invest in {symbol} (₹)", min_value=1000, step=1000)
            if st.button(f"Buy {symbol}"):
                if symbol not in st.session_state.portfolio:
                    st.session_state.portfolio[symbol] = {
                        'quantity': buy_amount / current_price,
                        'avg_price': current_price,
                        'current_price': current_price
                    }
                else:
                    existing = st.session_state.portfolio[symbol]
                    new_quantity = existing['quantity'] + (buy_amount / current_price)
                    new_avg = ((existing['quantity'] * existing['avg_price']) + buy_amount) / new_quantity
                    st.session_state.portfolio[symbol] = {
                        'quantity': new_quantity,
                        'avg_price': new_avg,
                        'current_price': current_price
                    }
                st.success(f"Bought ₹{buy_amount} worth of {symbol}")

        with col2:
            if symbol in st.session_state.portfolio:
                sell_pct = st.slider(f"Sell % of {symbol}", 0, 100, 25)
                if st.button(f"Sell {sell_pct}% of {symbol}"):
                    holding = st.session_state.portfolio[symbol]
                    sell_quantity = holding['quantity'] * (sell_pct / 100)
                    st.session_state.portfolio[symbol]['quantity'] -= sell_quantity
                    if st.session_state.portfolio[symbol]['quantity'] <= 0.001:  # Near zero
                        del st.session_state.portfolio[symbol]
                    st.success(f"Sold {sell_pct}% of {symbol} holding")

    # Display portfolio
    if st.session_state.portfolio:
        st.subheader("Your Portfolio")
        portfolio_df = pd.DataFrame.from_dict(st.session_state.portfolio, orient='index')
        portfolio_df['Investment'] = portfolio_df['quantity'] * portfolio_df['avg_price']
        portfolio_df['Current Value'] = portfolio_df['quantity'] * portfolio_df['current_price']
        portfolio_df['P&L'] = portfolio_df['Current Value'] - portfolio_df['Investment']
        portfolio_df['P&L %'] = (portfolio_df['P&L'] / portfolio_df['Investment']) * 100

        # Update prices (simulated)
        for symbol in st.session_state.portfolio:
            if 'df' in st.session_state and st.session_state.current_symbol == symbol:
                st.session_state.portfolio[symbol]['current_price'] = st.session_state.df['Close Price'].iloc[-1]

        st.dataframe(portfolio_df.style.applymap(
            lambda x: 'color: green' if x > 0 else 'color: red', subset=['P&L', 'P&L %']))

        total_investment = portfolio_df['Investment'].sum()
        total_value = portfolio_df['Current Value'].sum()
        total_pl = total_value - total_investment
        total_pl_pct = (total_pl / total_investment) * 100

        st.metric("Total Portfolio Value", f"₹{total_value:,.2f}",
                 delta=f"{total_pl_pct:.2f}% (₹{total_pl:,.2f})")
    else:
        st.info("Your portfolio is empty. Buy stocks to start tracking.")

# === Paper Trading Integration ===
with st.expander("📝 Paper Trading"):
    st.write("Practice trading with virtual money")

    if 'paper_balance' not in st.session_state:
        st.session_state.paper_balance = 100000
        st.session_state.paper_portfolio = {}
        st.session_state.paper_trades = []

    st.metric("Virtual Account Balance", f"₹{st.session_state.paper_balance:,.2f}")

    if 'df' in st.session_state and 'current_symbol' in st.session_state:
        symbol = st.session_state.current_symbol
        current_price = st.session_state.df['Close Price'].iloc[-1]

        col1, col2 = st.columns(2)
        with col1:
            paper_qty = st.number_input(f"Quantity of {symbol} to trade", min_value=1, step=1)
            paper_action = st.radio("Action", ["Buy", "Sell"], horizontal=True)

            if st.button("Execute Paper Trade"):
                cost = paper_qty * current_price

                if paper_action == "Buy":
                    if cost > st.session_state.paper_balance:
                        st.error("Insufficient virtual funds")
                    else:
                        st.session_state.paper_balance -= cost
                        if symbol not in st.session_state.paper_portfolio:
                            st.session_state.paper_portfolio[symbol] = paper_qty
                        else:
                            st.session_state.paper_portfolio[symbol] += paper_qty
                        st.session_state.paper_trades.append({
                            'date': datetime.now(),
                            'symbol': symbol,
                            'action': 'Buy',
                            'quantity': paper_qty,
                            'price': current_price,
                            'value': cost
                        })
                        st.success(f"Bought {paper_qty} shares of {symbol} at ₹{current_price:.2f}")

                elif paper_action == "Sell":
                    if symbol not in st.session_state.paper_portfolio or st.session_state.paper_portfolio[symbol] < paper_qty:
                        st.error(f"Not enough {symbol} shares to sell")
                    else:
                        st.session_state.paper_balance += cost
                        st.session_state.paper_portfolio[symbol] -= paper_qty
                        if st.session_state.paper_portfolio[symbol] <= 0:
                            del st.session_state.paper_portfolio[symbol]
                        st.session_state.paper_trades.append({
                            'date': datetime.now(),
                            'symbol': symbol,
                            'action': 'Sell',
                            'quantity': paper_qty,
                            'price': current_price,
                            'value': cost
                        })
                        st.success(f"Sold {paper_qty} shares of {symbol} at ₹{current_price:.2f}")

        with col2:
            if st.session_state.paper_portfolio:
                st.write("Current Holdings:")
                for sym, qty in st.session_state.paper_portfolio.items():
                    st.write(f"- {sym}: {qty} shares")
            else:
                st.info("No current holdings")

    if st.session_state.paper_trades:
        st.subheader("Trade History")
        st.dataframe(pd.DataFrame(st.session_state.paper_trades).sort_values('date', ascending=False))

# === ML, DL Model & Backtesting ===
if 'df' in st.session_state:
    df = st.session_state.df.copy()

    # Feature Engineering
    df['MA_20'] = df['Close Price'].rolling(window=20).mean()
    df['MA_50'] = df['Close Price'].rolling(window=50).mean()
    df['MA_200'] = df['Close Price'].rolling(window=200).mean()
    df['RSI'] = compute_rsi(df['Close Price'])
    df['MACD'], df['Signal_Line'] = compute_macd(df['Close Price'])
    df['Returns'] = df['Close Price'].pct_change()
    df['Target'] = (df['Close Price'].shift(-1) > df['Close Price']).astype(int)
    df.dropna(inplace=True)

    features = ['Open Price', 'High Price', 'Low Price', 'Close Price', 'Volume',
               'MA_20', 'MA_50', 'MA_200', 'RSI', 'MACD', 'Signal_Line', 'Returns']
    X = df[features]
    y = df['Target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # === Strategy Optimization with Genetic Algorithm ===
    with st.expander("🧬 Strategy Optimization"):
        st.write("Optimize strategy parameters using genetic algorithm")

        def evaluate_strategy(individual):
            """Evaluate strategy performance with given parameters"""
            # Individual contains: [ma_short, ma_long, rsi_low, rsi_high]
            ma_short, ma_long, rsi_low, rsi_high = individual

            # Simple strategy based on moving averages and RSI
            signals = []
            capital = 100000
            position = 0
            buy_price = 0

            for i in range(len(df)):
                if position == 0:
                    if df['MA_20'].iloc[i] > df['MA_50'].iloc[i] and df['RSI'].iloc[i] < rsi_low:
                        # Buy signal
                        position = capital // df['Close Price'].iloc[i]
                        buy_price = df['Close Price'].iloc[i]
                        capital -= position * buy_price
                        signals.append(1)
                    else:
                        signals.append(0)
                else:
                    if df['MA_20'].iloc[i] < df['MA_50'].iloc[i] and df['RSI'].iloc[i] > rsi_high:
                        # Sell signal
                        sell_price = df['Close Price'].iloc[i]
                        capital += position * sell_price
                        position = 0
                        signals.append(-1)
                    else:
                        signals.append(0)

            # Calculate fitness (final capital)
            return capital,

        if st.button("Run Genetic Optimization"):
            with st.spinner("Optimizing parameters..."):
                creator.create("FitnessMax", base.Fitness, weights=(1.0,))
                creator.create("Individual", list, fitness=creator.FitnessMax)

                toolbox = base.Toolbox()
                toolbox.register("attr_int", random.randint, 5, 50)
                toolbox.register("attr_float", random.uniform, 20, 80)
                toolbox.register("individual", tools.initCycle, creator.Individual,
                               (toolbox.attr_int, toolbox.attr_int, toolbox.attr_float, toolbox.attr_float), n=1)
                toolbox.register("population", tools.initRepeat, list, toolbox.individual)
                toolbox.register("mate", tools.cxBlend, alpha=0.5)
                toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
                toolbox.register("select", tools.selTournament, tournsize=3)
                toolbox.register("evaluate", evaluate_strategy)

                population = toolbox.population(n=10)
                algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=5, verbose=False)

                best_individual = tools.selBest(population, k=1)[0]
                st.success(f"Optimized Parameters: MA Short={best_individual[0]}, MA Long={best_individual[1]}, RSI Low={best_individual[2]:.1f}, RSI High={best_individual[3]:.1f}")

                # Plot optimization results
                fig, ax = plt.subplots()
                ax.plot([ind.fitness.values[0] for ind in population])
                ax.set_title("Fitness Improvement")
                ax.set_xlabel("Generation")
                ax.set_ylabel("Final Capital")
                st.pyplot(fig)

    # Machine Learning Model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    df['ML_Prediction'] = model.predict(X)

    # Deep Learning Model
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_seq = []
    y_seq = []
    for i in range(50, len(X_scaled)):
        X_seq.append(X_scaled[i-50:i])
        y_seq.append(y.iloc[i])
    X_seq, y_seq = np.array(X_seq), np.array(y_seq)

    X_train_dl, X_test_dl = X_seq[:int(0.8*len(X_seq))], X_seq[int(0.8*len(X_seq)):]
    y_train_dl, y_test_dl = y_seq[:int(0.8*len(y_seq))], y_seq[int(0.8*len(y_seq)):]

    lstm_model = Sequential()
    lstm_model.add(LSTM(50, return_sequences=True, input_shape=(X_seq.shape[1], X_seq.shape[2])))
    lstm_model.add(Dropout(0.2))
    lstm_model.add(LSTM(50))
    lstm_model.add(Dropout(0.2))
    lstm_model.add(Dense(1, activation='sigmoid'))
    lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    lstm_model.fit(X_train_dl, y_train_dl, epochs=5, batch_size=32, verbose=0)

    df['DL_Prediction'] = np.nan
    df.iloc[-len(y_seq):, df.columns.get_loc('DL_Prediction')] = (lstm_model.predict(X_seq) > 0.5).astype(int).flatten()

    # Multi-timeframe Analysis
    with st.expander("⏱ Multi-timeframe Analysis"):
        st.write("Compare different timeframes")

        if 'df' in st.session_state:
            # Simulate different timeframes by resampling
            timeframe_options = ['1H', '4H', '1D', '1W']
            selected_timeframes = st.multiselect("Select timeframes to compare", timeframe_options, ['1H', '4H'])

            fig = go.Figure()

            for tf in selected_timeframes:
                if tf == '1H':
                    resampled_df = df.resample('1H').last()
                    fig.add_trace(go.Scatter(x=resampled_df.index, y=resampled_df['Close Price'], name='1H'))
                elif tf == '4H':
                    resampled_df = df.resample('4H').last()
                    fig.add_trace(go.Scatter(x=resampled_df.index, y=resampled_df['Close Price'], name='4H'))
                elif tf == '1D':
                    resampled_df = df.resample('1D').last()
                    fig.add_trace(go.Scatter(x=resampled_df.index, y=resampled_df['Close Price'], name='1D'))
                elif tf == '1W':
                    resampled_df = df.resample('1W').last()
                    fig.add_trace(go.Scatter(x=resampled_df.index, y=resampled_df['Close Price'], name='1W'))

            fig.update_layout(title="Multi-timeframe Price Comparison", xaxis_title="Date", yaxis_title="Price")
            st.plotly_chart(fig, use_container_width=True)

    # Risk Management Tools
    with st.expander("🛡 Risk Management Tools"):
        st.subheader("Position Sizing Calculator")

        col1, col2 = st.columns(2)
        account_size = col1.number_input("Account Size (₹)", min_value=1000, value=100000, step=1000)
        risk_percent = col2.number_input("Risk per Trade (%)", min_value=0.1, max_value=10.0, value=1.0, step=0.1)

        if 'df' in st.session_state:
            current_price = df['Close Price'].iloc[-1]
            stop_loss_pct = st.slider("Stop Loss (%)", min_value=0.1, max_value=20.0, value=2.0, step=0.1)

            risk_amount = account_size * (risk_percent / 100)
            stop_loss_price = current_price * (1 - (stop_loss_pct / 100))
            position_size = risk_amount / (current_price - stop_loss_price)

            st.metric("Position Size", f"{position_size:.2f} shares")
            st.metric("Investment Amount", f"₹{position_size * current_price:,.2f}")
            st.metric("Potential Loss", f"₹{risk_amount:,.2f} ({risk_percent}% of account)")

        st.subheader("Stop Loss Optimizer")
        if 'df' in st.session_state:
            lookback = st.slider("Lookback Period (days)", 5, 100, 20)
            atr_period = st.slider("ATR Period", 5, 20, 14)

            # Calculate ATR
            high_low = df['High Price'] - df['Low Price']
            high_close = np.abs(df['High Price'] - df['Close Price'].shift())
            low_close = np.abs(df['Low Price'] - df['Close Price'].shift())
            ranges = pd.concat([high_low, high_close, low_close], axis=1)
            true_range = np.max(ranges, axis=1)
            atr = true_range.rolling(atr_period).mean()

            # Suggested stop loss
            suggested_sl = df['Close Price'].iloc[-1] - (2 * atr.iloc[-1])
            sl_pct = ((df['Close Price'].iloc[-1] - suggested_sl) / df['Close Price'].iloc[-1]) * 100

            st.metric("Suggested Stop Loss Price", f"₹{suggested_sl:.2f}")
            st.metric("Stop Loss Percentage", f"{sl_pct:.2f}%")

            fig, ax = plt.subplots()
            ax.plot(df.index[-lookback:], df['Close Price'].iloc[-lookback:], label='Price')
            ax.axhline(y=suggested_sl, color='r', linestyle='--', label='Suggested Stop Loss')
            ax.legend()
            st.pyplot(fig)

    # Backtest Strategy
    st.subheader("🧪 Strategy Backtest (ML-based)")
    position = 0
    capital = 100000
    buy_price = 0
    trades = []
    for i, row in df.iterrows():
        if position == 0 and row['ML_Prediction'] == 1:
            position = capital // row['Close Price']
            buy_price = row['Close Price']
            capital -= position * buy_price
            trades.append((i, buy_price, 'Buy'))
        elif position > 0 and row['ML_Prediction'] == 0:
            sell_price = row['Close Price']
            capital += position * sell_price
            trades.append((i, sell_price, 'Sell'))
            position = 0

    st.write(f"💰 Final Capital: ₹{capital:.2f}")
    trade_df = pd.DataFrame(trades, columns=["Time", "Price", "Action"])
    st.dataframe(trade_df.tail(10))

    # Plot
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(df.index, df['Close Price'], label='Close Price')
    for trade in trades:
        color = 'g' if trade[2] == 'Buy' else 'r'
        ax.scatter(trade[0], trade[1], color=color, label=trade[2], s=50)
    ax.set_title("Price Chart with Trades")
    ax.legend()
    st.pyplot(fig)

    # Social Features
    with st.expander("👥 Community Features"):
        st.subheader("Share Your Trade")

        if len(trades) > 0:
            last_trade = trades[-1]
            trade_note = st.text_area("Add a note about your trade")

            if st.button("Share to Community"):
                st.session_state.community_trades.append({
                    'user': "You",  # In a real app, use actual user ID
                    'symbol': st.session_state.current_symbol,
                    'action': last_trade[2],
                    'price': last_trade[1],
                    'time': last_trade[0],
                    'note': trade_note
                })
                st.success("Trade shared with community!")

        st.subheader("Community Trades")
        if st.session_state.community_trades:
            community_df = pd.DataFrame(st.session_state.community_trades)
            st.dataframe(community_df.sort_values('time', ascending=False))
        else:
            st.info("No community trades yet. Be the first to share!")

    # Telegram Alerts
    st.subheader("🔔 Alerts via Telegram")
    telegram_token = st.text_input("Bot Token", type="password")
    chat_id = st.text_input("Chat ID")
    if st.button("Send Alert for DL Prediction"):
        pred_dl = df.iloc[-1]['DL_Prediction']
        signal = "UP 📈" if pred_dl == 1 else "DOWN 📉"
        msg = f"Deep Learning Signal for Next Candle: {signal}"
        if telegram_token and chat_id:
            requests.get(f"https://api.telegram.org/bot{telegram_token}/sendMessage?chat_id={chat_id}&text={msg}")
            st.success("📤 Telegram Alert Sent!")

    # Auto-refresh toggle
    if st.checkbox("🔄 Enable Live Refresh (every 30s)"):
        time.sleep(30)
        st.experimental_rerun()
else:
    st.info("Upload or fetch data to proceed.")

# Helper functions
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def compute_macd(series, fast=12, slow=26, signal=9):
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line
'''

In [ ]:
!pip install streamlit pandas scikit-learn matplotlib pyotp tensorflow smartapi-python requests
!pip install logzero
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [ ]:
with open("dashboardert.py", "w") as f:
    f.write(code)

In [ ]:
!pip install streamlit pandas numpy scikit-learn matplotlib pyotp requests tensorflow smartapi-python deap textblob plotly yfinance
!python -m textblob.download_corpora  # For sentiment analysis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 2.9 MB/s eta 0:00:00
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [ ]:

from pyngrok import ngrok

# Replace this with your own authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("2v1WPxUy5A88vovYO0vTqkiwtf6_4dDm4btDEeocT8S9ZaTBU")

# Correct port connection (without quotes)
public_url = ngrok.connect(8501)
print("🔗 Streamlit App is Live at:", public_url)



🔗 Streamlit App is Live at: NgrokTunnel: "https://8d56-34-80-215-160.ngrok-free.app" -> "http://localhost:8501"


In [ ]:

!pkill -f streamlit
!pkill -f ngrok

In [ ]:
!streamlit run dashboardert.py &>/content/log.txt &
public_url = ngrok.connect(8501)
print(f"🚀 Streamlit app is live at: {public_url}")

🚀 Streamlit app is live at: NgrokTunnel: "https://4b33-34-80-215-160.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install streamlit plotly yfinance textblob
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [ ]:
!pip install streamlit-aggrid

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 28.6 MB/s eta 0:00:00
